In [ ]:
# All the Required Import statements
import gc
import tensorflow
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, AveragePooling2D, BatchNormalization, add
from tensorflow.keras import Input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.regularizers import l2 as L2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model
from PIL import Image
import pandas
import numpy

In [ ]:
# Collect Unrefereced Memory and Free them using gc.collect()
gc.collect()

# Hyperparameters
url = '/kaggle/input/aptos2019-blindness-detection/'

shape = (224, 224)
rotateAngle = 15

In [ ]:
train_data = pandas.read_csv(url + '/train.csv')
test_data = pandas.read_csv(url + '/test.csv')

train_y = train_data['diagnosis'].astype('category', [0, 1, 2, 3, 4])
train_y = pandas.get_dummies(train_y).values

# print(train_y)

train_y[:,3] = numpy.where(train_y[:,4] > train_y[:,3], train_y[:,4], train_y[:,3])
train_y[:,2] = numpy.where(train_y[:,3] > train_y[:,2], train_y[:,3], train_y[:,2])
train_y[:,1] = numpy.where(train_y[:,2] > train_y[:,1], train_y[:,2], train_y[:,1])
train_y[:,0] = numpy.where(train_y[:,1] > train_y[:,0], train_y[:,1], train_y[:,0])

In [ ]:
print(train_y)

In [ ]:
# This Method Splits the data depending on which set you want to choose from (i.e, train or test)
# and giving a start and end point will give you the data required only upto that amount
def split_data(name, start=0, end=-1, rotation=0):

    if name == 'train':
        data = train_data

    elif name == 'test':
        data = test_data

    else:
        print('wrong split name')
        return None

    if end > data.shape[0] or end == -1:
        end = data.shape[0]

    if rotation >= 360:
        rotation %= 360

    x = []

    for i in range(start, end):
        image = Image.open(url + '/' + name + '_images/' +
                           data['id_code'][i] + '.png').convert('L')
        image = image.resize(shape)
        image = image.rotate(rotation)
        x.append(numpy.asarray(image).reshape(shape + (1,)))
    x = numpy.array(x)

    if name == 'train':
        return x, train_y[start:end]

    elif name == 'test':
        return x

In [ ]:
def xception_model():
    
    model = Xception(include_top = False, weights = None, input_shape = shape + (1,), pooling = 'avg', classes = False)
    x = model.output
    
    x = Dense(1024, activation = 'relu')(x)
    x = Dense(256, activation = 'relu')(x)
    outputs = Dense(5, activation = 'sigmoid')(x)
    
    model = Model(model.input, outputs)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    print(model.summary())
    
    return model

In [ ]:
# Make methods to create new model
# This method creates a basic model
def custom_model():

    inputs = Input(shape + (1,))

    x = BatchNormalization()(inputs)

    x = Conv2D(32, (5, 5), 2, data_format = 'channels_last',activation = 'relu')(x)
    x = MaxPool2D(strides = 2)(x)
    
    x = Conv2D(32, (3, 3), activation = 'relu')(x)
    y = Dropout(0.25)(x)

    for _ in range(2):
        x = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(y)
        x = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(x)
        x = Dropout(0.1)(x)

        y = add([y, x])
        
    y = Conv2D(64, (3, 3), activation = 'relu')(x)

    for _ in range(2):
        x = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(y)
        x = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(x)
        x = Dropout(0.1)(x)

        y = add([y, x])
        
    y = Conv2D(128, (3, 3), activation = 'relu')(x)

    for _ in range(3):
        x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(y)
        x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
        x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
        x = Dropout(0.1)(x)

        y = add([y, x])
    
    x = AveragePooling2D()(y)

    x = Flatten()(x)
    x = Dense(1000, activation = 'relu')(x)
    x = Dense(500, activation = 'relu')(x)
    x = Dense(20, activation = 'relu')(x)
    outputs = Dense(5, activation = 'sigmoid')(x)

    model = Model(inputs,outputs)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    print(model.summary())
    
    return model

In [ ]:
def resnet_model():
    model = ResNet50(include_top = False, weights = None, input_shape = (shape + (1,)), pooling = 'avg')
    x = model.output
    
    x = Flatten()(x)
    x = Dense(1024, activation = 'relu', kernel_regularizer = L2(0.01), bias_regularizer = L2(0.01))(x)
    x = Dense(256, activation = 'relu', kernel_regularizer = L2(0.01), bias_regularizer = L2(0.01))(x)
    outputs = Dense(5, activation = 'sigmoid')(x)
    
    model = Model(model.input, outputs)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    print(model.summary())

    return model

In [ ]:
def conv_model():

    inputs = Input(shape + (1,))

    x = BatchNormalization()(inputs)
    x = Conv2D(512, (3, 3), data_format='channels_last',
               activation='relu')(x)
    x = MaxPool2D((3, 3))(x)

    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(5, activation='relu')(x)

    model = Model(inputs, outputs)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    print(model.summary())

    return model

In [ ]:
# Important
# -> This Method fits the model calling the fit method multiple times
# -> It uses manual batch system and manual epoch system rather than passing the arguments in fit method
#    This is done due to large images (each image is around 3000x3000 on an average around 1800x1800)
#    and the current hardware wouldn't be able to store this large amount of data.
# -> We call the Fit method (batch_size * epochs) times.
# -> We call the Fit method multiple times to meet the required Hardware Limitations
def model_fitting(model, batch_size=250, epochs=5, validation=False):

    early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, patience=3, verbose=1, mode='auto')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto', verbose=1)
    
    end = int(train_data.shape[0] * 0.8) if validation else train_data.shape[0]

#     for i in range(epochs):
#         print("\n\nEPOCHS (" + str(i + 1) + "/" + str(epochs) + ") : ")
    with tensorflow.device('/device:GPU:0'):
        for j in range(0, end, batch_size):

            last = end if (j + batch_size) > end else (j + batch_size)

            for k in range(0,360,rotateAngle):

                print("Rotating Image by",k,"degrees.")

                train_x, train_y = split_data('train', j, last, k)
                model.fit(train_x, train_y, batch_size=50, epochs=epochs, shuffle=True, callbacks = [early_stop, reduce_lr])

                del train_x, train_y
                gc.collect()

            print("(" + str(last) + "/" + str(end) + ") images have been Scanned!!")

        if validation:
            validate_x, validate_y = split_data('train', end)
            model.evaluate(validate_x, validate_y)

            del validate_x, validate_y
            gc.collect()

        gc.collect()

    return model



In [ ]:
# Submitting the predicted values can be done using this method
# Since the test_y values are not given we have to submit the model to check for the accuracy achieved
def submit(model, batch_size=300):

    gc.collect()

    for i in range(0, test_data.shape[0], batch_size):

        end = test_data.shape[0] if (i + batch_size) > test_data.shape[0] else (i + batch_size)

        test_x = split_data('test', i, end)
        test_y = model.predict(test_x)

        test_y = numpy.sum((test_y > 0.5).astype(int), axis = 1) - 1

        predicted_data = pandas.DataFrame({'id_code': test_data['id_code'][i:end].values, 'diagnosis': test_y})

        if i == 0:
            final_data = predicted_data

        if i != 0:
            final_data = final_data.append(predicted_data, ignore_index=True)

        del test_x, test_y
        gc.collect()

    final_data[['id_code', 'diagnosis']].to_csv('submission.csv', index=False)
    print('Data Converted to CSV Format')

    return final_data[['id_code', 'diagnosis']]

In [ ]:
if __name__ == '__main__':

    print('Images are resized to', shape, '\n\n')

#     model = conv_model()
#     model = custom_model()
    model = resnet_model()
#     model = xception_model()
    model = model_fitting(model, batch_size=750, epochs=25, validation=False)
    model.save('custom_model')
    print(submit(model))